In [1]:
from bs4 import BeautifulSoup
import requests
import json


def findEan(url):
    if not url:
        print("not url")
        return;
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        ean_element = soup.find('script', {'data-section-type': 'static-product'})
        name = soup.find('h1', {'class': 'product-title'})
        if ean_element:
            script_content = json.loads(ean_element.string)
            # Variants listesindeki her bir varyantın barcode değerini al
            variants = script_content.get('product').get("variants", [])
            for variant in variants:
                barcode = variant.get("barcode")
                if barcode:
                    # print("Barcode:", barcode)
                    return barcode 
            return "No barcode found"
        else:
            print("Script element with data-section-type 'static-product' not found.")
            return "No script element found"
    else:
        print("Failed to retrieve content from the URL.")
        return "Request failed"


/Users/alidoner/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [18]:
import requests,csv
from bs4 import BeautifulSoup



# Ana URL
# base_url = 'https://www.kappersleverancier.nl/'
# url = 'https://www.kappersleverancier.nl/collections/haarstyling'
# url ='https://www.kappersleverancier.nl/collections/salon-apparatuur'




# Ürün listesi için boş liste oluştur
products = []

def findAllItem(url):
 # Headers ekleyin
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    # Tüm ürünlerin bulunduğu ana sayfa
    base_url = 'https://www.kappersleverancier.nl/'

    # Tüm sayfaları dolaşmak için bir döngü
        # URL'den sayfa içeriğini getir 
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        product_elements = soup.findAll('div', {'class' : 'productitem'})
        for item in product_elements:
            if not item:
                 continue
            else:
                name = item.find('h2',{'class':'productitem--title'})
                money = item.find('span',{'class':'money PRICE-Money'})
                image = item.find('img',{'class':'productitem--image-primary'})["src"]
                if image.startswith('//'):
                        image = image[2:]
                # print(money.text.strip())
                href = item.find('a',{'class':'productitem--image-link'}).get('href')
                if not href:
                    print("yok " + url)
                products.append({
                        'name': name.text.strip(),
                        'price': money.text.strip(),
                        'link': href,
                        'image':image
                    })
        print(len(products))
        try:
            hasNextPagina = soup.find('li',{'class':'pagination--next'}).find('a',{'class':'pagination--item'}).get('href')
                
            return(products.append(findAllItem(base_url+hasNextPagina)))
                
        except:
            return products
        
        # print(product_elements)

        # print(base_url+ product_elements.find('a',{'class':'productitem--image-link'}).get('href'))

            # for product_elem in product_elements:
            #     # Ürün adını ve bağlantısını alın
            #     product_name = product_elem.find('h2', class_='product-title').text.strip()
            #     product_link = base_url + product_elem.find('a', class_='product-card').get('href')
                
            #     # Ürün fiyatını alın
            #     product_price = product_elem.find('span', class_='product-price').text.strip()
                
            #     # Ürün bilgilerini listeye ekleyin
            #     products.append({
            #         'name': product_name,
            #         'price': product_price,
            #         'link': product_link
            #     })

            

            # Ürünlerin bulunduğu bölümü belirleyin

   

In [20]:
import requests
from bs4 import BeautifulSoup

# Hedef URL
base_url = 'https://www.kappersleverancier.nl'

# Headers ekleyin
headers = {
    
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
# Ürün listesi için boş liste oluştur
products = []
# URL'den sayfa içeriğini getir
response = requests.get(base_url, headers=headers)
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Ana menü öğelerini bul
    main_menu = soup.find('nav', {'class': 'site-navigation'})
    menu_links = [base_url + a['href'] for a in main_menu.find_all('a', href=True) if a['href'].startswith('/')]
    
    for a in menu_links:
        # print("link + " , a)
        products.append(findAllItem(a))
    # products.append(findAllItem(menu_links[1]))
    
    file = open("producten.csv",'w',newline='',)
    writer = csv.writer(file)
    writer.writerow(['EAN', 'Name', 'Price', 'Link','image'])  # Başlık satırı ekle
    for product in products:

        if product is not  None:
                try:
                    
                    ean = findEan(base_url + product['link'])
                    print("geldi" + ean)

                    writer.writerow([ean, product['name'], product['price'], base_url + product['link'],product['image']])
                except:
                     continue


    file.close()     
else:
    print("Failed to retrieve the main page")


30
79
127
132
183
231
264
300
322
336
385
409
413
429
478
526
542
567
616
619
669
693
704
753
801
849
897
936
989
1037
1085
1117
1169
1170
1192
1217
1235
1265
1314
1321
1371
1419
1461
1468
1471
1519
1535
1584
1602
1643
1668
1696
1701
1709
1758
1806
1825
1876
1924
1972
2020
2068
2090
2126
2166
2209
2234
2283
2287
2316
2331
2344
2393
2441
2460
2463
2464
2513
2561
2609
2657
2705
2753
2801
2849
2868
2877
geldi8021660018787
geldi8021660019371
geldi8021660016783
geldi8021660018480
geldiNo barcode found
geldi8682035087588
geldi8021660018794
geldi840302411407
geldi8681701001798
geldi8682035088004
geldi8445000019140
geldi8445000021655
geldi8445000016163
geldi8445000016163
geldi8445000019003
geldiNo barcode found
geldi4016187057924
geldi8445000028876
geldi8445000027107
geldi4016187042784
geldiNo barcode found
geldi4016187004676
geldiNo barcode found
geldiNo barcode found
geldiNo barcode found
geldiNo barcode found
geldi8445000012851
geldi8445000022911
geldi8445000026148
geldi8445000026872
geldi8